In [17]:
import pandas as pd
import numpy as np
import sys
import copy
import time

data=pd.DataFrame([[np.NaN]*9]*9)

In [18]:
#I'll optimize the algorithm in the future.
#default data
data=pd.DataFrame(
                      [[5,3,0,0,7,0,0,0,0],
                       [6,0,0,1,9,5,0,0,0],
                       [0,9,8,0,0,0,0,6,0],
                       [8,0,0,0,6,0,0,0,3],
                       [4,0,0,8,0,3,0,0,1],
                       [7,0,0,0,2,0,0,0,6],
                       [0,6,0,0,0,0,2,8,0],
                       [0,0,0,4,1,9,0,0,5],
                       [0,0,0,0,8,0,0,7,9]])

In [19]:
#create backpack (only do once)
backpack=[]
for i in range(9):
    backpack.append([0]*9)

for i in range(9):
    for j in range(9):
        if(data.iloc[i,j] == 0):
            x=int(i/3)*3
            y=int(j/3)*3
            backpack[i][j]= list(set(data.iloc[i].append(data.iloc[:,j]).append(data.iloc[ x, y:y+3 ])
                                                                                 .append(data.iloc[ x+1 , y:y+3 ])
                                                                                 .append(data.iloc[ x+2 , y:y+3])) ^ set([1,2,3,4,5,6,7,8,9,0]))
        else:
            backpack[i][j]=[]


In [20]:
def mainfunction():
    global data; global backpack
    for i in range(9):
        for j in range(9):
            if(data.iloc[i,j]!=0):
                continue
            
            if(len(backpack[i][j])==1):
                #print("Filling position {} because of only 1 candidate with {}.".format((i,j),backpack[i][j][0]))
                data.iloc[i,j]=backpack[i][j][0]
                revise(i,j,backpack[i][j][0])
                #print(backpack)
                return 0
            
            rowst=[]
            colst=[]
            sqlst=[]
            rowst=backpack[i][:j] + backpack[i][j+1:] #加入行(已剔除)
            
            for m in range(9):#加入列（已剔除）
                if(m==i):
                    continue
                colst.append(backpack[m][j])
                
            x=int(i/3)*3#加入3x3块（已剔除）
            y=int(j/3)*3
            sqlst=backpack[x][y:y+3]+backpack[x+1][y:y+3]+backpack[x+2][y:y+3]
            sqlst.remove(backpack[i][j])

            for number in backpack[i][j]:
                if (  (str(number) not in str(rowst))  or (str(number) not in str(colst))  or (str(number) not in str(sqlst))  ):
                    #print("Filling {} with {} because of unique".format((i,j),number))
                    data.iloc[i,j]=number
                    revise(i,j,number)
                    return(0)

                
    return -1



In [21]:
def revise(i,j,number):
    global backpack
    backpack[i][j]=[]
    for m in range(9):
        try:
            backpack[i][m].remove(number)
        except:
            pass
        
        try:
            backpack[m][j].remove(number)
        except:
            pass
        
        x=int(i/3)*3
        y=int(j/3)*3
        
    try:
        backpack[x][y].remove(number)
    except:
            pass
    try:
        backpack[x+1][y].remove(number)    
    except:
            pass
    try:
        backpack[x+2][y].remove(number)    
    except:
            pass
    try:
        backpack[x][y+1].remove(number)    
    except:
            pass
    try:
        backpack[x+1][y+1].remove(number)    
    except:
            pass
    try:
        backpack[x+2][y+1].remove(number)    
    except:
            pass
    try:
        backpack[x][y+2].remove(number)    
    except:
            pass
    try:
        backpack[x+1][y+2].remove(number)    
    except:
            pass    
    try:
        backpack[x+2][y+2].remove(number)    
    except:
            pass

        
        

In [22]:
def callme():
    while(1):
        i=mainfunction()
        if(i==-1):
            #print("Data Entered.")
            break

In [23]:
def random_try():
    global data; global backpack
    data_backup=data.copy()
    backpack_backup=copy.deepcopy(backpack)
    #find the shortest backpack list
    x=-1;y=-1;length=10
    for i in range(9):
        for j in range(9):
            if(data.iloc[i,j]!=0):
                continue
            if(len(backpack[i][j])<length):
                x=i;y=j;length=len(backpack[i][j])    
    tmp=backpack[x][y]
    
    for item in tmp:
        #print("----------------------------------------------\nBefore random guess, Now data is:\n {}\n----------------------------------------------".format(data))
        #print("----------------------------------------------\nBefore random guess, Now backpack is:\n {}\n----------------------------------------------".format(backpack))        
        #print("Candidate guessing numbers at: {} are: {}".format((x,y),tmp))  
        #print("Trying to fill position: {}  with: {}  from  {}".format((x,y),item,tmp))
        flag=0
        data.iloc[x,y]=item
        revise(x,y,item)          #backpack 在此被改过了        
        
        
        #print("---------------ADDED BUT BEFORE ENTERING OTHERS---------------\nData:\n{}\n    Backpack:\n{}\n------------------------------".format(data,backpack))
        callme() #这个函数修改了我的back_up们！！！！！！！！！！！！！！！
        #print("---------------AFTER---------------\nData:\n{}\n    Backpack:\n{}\n------------------------------".format(data,backpack))
        
        
        if(data.sum().sum()==405):
            #print("All Done.")
            time_end=time.time()
            print("--------------------")
            print(data)
            print("--------------------")
            print('Successfully Solved.\nTotal time: {} seconds.'.format(time_end-time_start))
            sys.exit(1)
        #如果完成了，返回给最外层调用函数：返回值 1
        
        #judge wrong number
        #print("Before judging wrong number:")
        #print(data)
        #print(backpack)
        #print("^^^^^^^^^^^^\ndata_backup:\n{}\nbackpack_backup:\n{}\n^^^^^^^^^^^^".format(data_backup,backpack_backup))
        for i in range(9):
            for j in range(9):
                if((len(backpack[i][j])==0) and (data.iloc[i,j]==0)):     
                    #print("(X) Random guess at position: {}  with: {}".format((x,y),item))
                    
                    flag=1
                    break
            if(flag==1):
                break
        #judge wrong number done. if is wrong, flag==1        
        if(flag==1):
            #print("Filling position: {}  with: {} is wrong.".format((x,y),item))
            #print("Back_ups are:\ndata:\n{}\nbackpack:\n{}".format(data_backup,backpack_backup))
            data=data_backup.copy()
            backpack=copy.deepcopy(backpack_backup)
            #print("Reversed Data to {}\n############".format(data))
            #print("Reversed Backpack to {}\n############".format(backpack))
            continue
        if(flag==0):
            #print("Further guessing...")
            random_try_return=random_try()
            if(random_try_return==-1):#该不该删掉下面的两行？
                data=data_backup.copy()
                backpack=copy.deepcopy(backpack_backup)
                continue
                
                
                
    #如果几次猜测都是错的，返回-1，返回给上一层的猜测数，那个猜测数是错的
    #print("All the guessings: {} are wrong.".format(tmp))
    #data=data_backup.copy()
    #backpack=copy.deepcopy(backpack_backup)
    #print("Reversed Data to {}\n############".format(data))
    #print("Reversed Backpack to {}\n############".format(backpack))
    return -1
        
                    
            
#random guess again

In [24]:
while(1):
    time_start=time.time()
    callme()
    if(data.sum().sum()==405):
        time_end=time.time()
        print("--------------------")
        print(data)
        print("--------------------")
        print('Successfully Solved.\nTotal time: {} seconds.'.format(time_end-time_start))
        break
    else:
        res=random_try()#等于1时候，说明数独做好了
    if(res==1):
        print("SUCCESS")
        break
    if(res==-1):
        print("Fatal problem occured: All tests were wrong.\nThis Suduku has no answer.")
        print(data)
        sys.exit(0)

    


--------------------
   0  1  2  3  4  5  6  7  8
0  5  3  4  6  7  8  9  1  2
1  6  7  2  1  9  5  3  4  8
2  1  9  8  3  4  2  5  6  7
3  8  5  9  7  6  1  4  2  3
4  4  2  6  8  5  3  7  9  1
5  7  1  3  9  2  4  8  5  6
6  9  6  1  5  3  7  2  8  4
7  2  8  7  4  1  9  6  3  5
8  3  4  5  2  8  6  1  7  9
--------------------
Successfully Solved.
Total time: 0.109375 seconds.
